In [22]:
import torch
from d2l import torch as d2l



def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [30]:
X = torch.tensor([[[0.,1.,2.],
                   [3.,4.,5.],
                   [6.,7.,8.]],
                   [[1.,2.,3.],
                    [4.,5.,6.],
                    [7.,8.,9.]]])
K = torch.tensor([[[0.,1.],
                   [2.,3.]],
                   [[1.,2.],
                    [3.,4.]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [31]:
# 多输出通道
def corr2d_multi_in_out(X, K):
     # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0) #大X

# 多个通道的输入先和一个核卷积，求和得到一层输出，然后输入和下一层的卷积核卷积，……

In [32]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [33]:

corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [36]:
# 1 X 1
def corr2d_multi_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    # 全连接层的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [40]:
X = torch.normal(0.,1.,(3,3,3))
K = torch.normal(0.,1.,(2,3,1,1))

Y1 = corr2d_multi_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1-Y2).sum())<1e-6